# Create a Mockup Dataset

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Create a date range for the dataset
start_date = datetime(2020, 1, 1)
end_date = datetime(2021, 12, 31)
date_range = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]

# Create an empty DataFrame to store the mock-up dataset
data = {
    "Month": [],  # Changed from "Date" to "Month"
    "Seasonality": [],
    "MarketTrends": [],
    "Temperature": [],
    "CompetitorPrices": [],
    "Demand": []
}

# Generate data for each date in the date range
num_samples = 25000  # Number of rows in the dataset
for _ in range(num_samples):
    date = np.random.choice(date_range)  # Randomly select a date
    seasonality = np.sin(2 * np.pi * date.month / 12)  # Simulated seasonal pattern
    market_trends = seasonality + np.random.uniform(-0.1, 0.1)  # Simulated market trends with noise
    temperature = np.random.uniform(10, 30)  # Simulated temperature data
    competitor_prices = 100 + seasonality * 50 + np.random.normal(0, 10)  # Simulated competitor prices
    
    # Create a simple linear relationship with demand
    demand = 200 - (2 * seasonality) + (3 * market_trends) + (4 * temperature) + (1.5 * competitor_prices) + np.random.normal(0, 20)
    
    # Extract the month from the date
    month = date.month
    
    data["Month"].append(month)
    data["Seasonality"].append(seasonality)
    data["MarketTrends"].append(market_trends)
    data["Temperature"].append(temperature)
    data["CompetitorPrices"].append(competitor_prices)
    data["Demand"].append(demand)

# Create the DataFrame
mockup_dataset = pd.DataFrame(data)

# Save the dataset to a CSV file
mockup_dataset.to_csv("demand_data_25000.csv", index=False)

# Load the demand data from the CSV file
data = pd.read_csv("demand_data_25000.csv")
data

,Month,Seasonality,MarketTrends,Temperature,CompetitorPrices,Demand
0,2,8.660254e-01,0.909301,13.926607,144.374681,468.431790
1,10,-8.660254e-01,-0.914896,10.372791,52.738700,289.880097
2,3,1.000000e+00,1.025157,11.768730,157.259648,451.834077
3,7,-5.000000e-01,-0.484012,21.669045,78.885405,388.767641
4,10,-8.660254e-01,-0.891304,12.872255,42.654156,326.549231
...,...,...,...,...,...,...
24995,7,-5.000000e-01,-0.556287,12.809886,78.027479,378.487466
24996,7,-5.000000e-01,-0.516772,28.170018,73.818853,389.655374
24997,6,1.224647e-16,-0.029609,12.061932,85.930490,390.769711
24998,7,-5.000000e-01,-0.406749,14.057442,105.174492,427.675497


# Create a feedforward neural network model to predict the target demand given the features input

In [2]:
from sklearn.preprocessing import StandardScaler
# Define the target variable (Demand) and features, including 'Month'
target = "Demand"
features = ["Month", "Seasonality", "MarketTrends", "Temperature", "CompetitorPrices"]

# Split the data into features (X) and target variable (y)
X = data[features].values
y = data[target].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (recommended for neural networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Define a simple feedforward neural network model
class FeedForwardNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Initialize the neural network model
input_size = len(features)
hidden_size = 64
output_size = 1
model = FeedForwardNN(input_size, hidden_size, output_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create DataLoader for batch training (optional but recommended for large datasets)
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.view(-1, 1))  # Reshape targets for MSE loss
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)
    y_pred = y_pred_tensor.numpy()

# Convert predictions and ground truth to numpy arrays
y_test_np = y_test_tensor.numpy()

# Evaluate the model
mae = mean_absolute_error(y_test_np, y_pred)
r2 = r2_score(y_test_np, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R²) Score: {r2:.4f}")

Mean Absolute Error (MAE): 15.89
R-squared (R²) Score: 0.8997


# Given random inputs of the features, use the model to predict the forecasted Demand (TEU)

In [3]:
import random
# Generate demand forecasts for each month using the trained model
forecasted_demand = []
for month in range(1, 13):
    input_data = torch.tensor([[month] + [random.uniform(-1, 1), random.uniform(-0.1, 0.1), random.uniform(10, 30), random.uniform(50, 150)]], dtype=torch.float32)
    forecast = model(input_data).item()
    forecasted_demand.append(forecast)

# Create a DataFrame for the demand forecasts
months = list(range(1, 13))
demand_df = pd.DataFrame({"Month": months, "Forecasted Demand (TEU)": forecasted_demand})
print(demand_df)

    Month  Forecasted Demand (TEU)
0       1              7821.809082
1       2              6005.863770
2       3              6473.661621
3       4              4521.659668
4       5              3573.890381
5       6              6178.895996
6       7              6755.963867
7       8              6327.354004
8       9              7834.688965
9      10              4319.589355
10     11              7790.633301
11     12              7416.341309


# Create a mockup dataset for the supply_data

In [31]:
import pandas as pd
import random

# Number of containers
num_containers = 190

# Create a list of container IDs (e.g., C001, C002, ...)
container_ids = ["C{:03d}".format(i) for i in range(1, num_containers + 1)]

# Generate random capacities for the containers (e.g., between 20 and 100 TEU)
capacities = [random.randint(20, 100) for _ in range(num_containers)]

# Generate random shipping status for the containers (0 for not shipping, 1 for shipping)
shipping_status = [random.choice([0, 1]) for _ in range(num_containers)]

# Create a DataFrame for the supply dataset
supply_data = {
    "ContainerID": container_ids,
    "Capacity (TEU)": capacities,
    "Shipping Status": shipping_status
}

supply_df = pd.DataFrame(supply_data)
supply_df

,ContainerID,Capacity (TEU),Shipping Status
0,C001,74,0
1,C002,49,1
2,C003,40,1
3,C004,56,0
4,C005,31,0
...,...,...,...
185,C186,23,1
186,C187,32,0
187,C188,48,1
188,C189,20,0


# 1.) Input the features and get the forecasted demand for that month.
# 2.) Based on the forecasted demand, allocate resourses to the different containers each day of the month

In [46]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler

# Create a dictionary to map month names to their numbers
month_numbers = {
    1: "Jan",
    2: "Feb",
    3: "Mar",
    4: "Apr",
    5: "May",
    6: "Jun",
    7: "Jul",
    8: "Aug",
    9: "Sep",
    10: "Oct",
    11: "Nov",
    12: "Dec"
}

# Create a dictionary to track the time since allocation for each container
container_allocation_time = {}

# Define a function to perform allocation and forecasting for a specific month
def allocate_and_forecast(month, seasonality, market_trends, temperature, competitor_prices):
    # Get the month number from its name
    month_num = int(month)
    
    # Check if the month number is valid
    if month_num < 1 or month_num > 12:
        print("Invalid month input. Please enter a number between 1 and 12.")
        return
    
    # Convert input features to a PyTorch tensor
    input_data = torch.tensor([[month_num, seasonality, market_trends, temperature, competitor_prices]], dtype=torch.float32)
    
    # Use the trained model to forecast demand for the specified month
    forecasted_demand = model(input_data).item()
    print(f"Forecasted Demand for {month}: {forecasted_demand:.2f} TEU")
    
    # Create a copy of the supply dataset for this month
    allocation_df = supply_df.copy()
    allocation_df["Allocated (TEU)"] = 0  # Initialize the allocated column

    # Filter containers to include only those with Shipping Status equal to 1
    eligible_containers = allocation_df[allocation_df["Shipping Status"] == 1]
    
    # Shuffle the eligible containers randomly for each month
    eligible_containers = eligible_containers.sample(frac=1)
    
    # Calculate daily allocation
    daily_allocation = forecasted_demand / 31  # Divide monthly demand by the number of days in the month
    
    allocated_container_ids_month = []
    
    for day in range(1, 32):
        
        # Reset daily demand to the calculated daily allocation
        daily_demand = daily_allocation
        
        # Filter out already allocated containers from eligible containers
        eligible_containers = eligible_containers[~eligible_containers["ContainerID"].isin(allocated_container_ids_month)]
        
        # Remove containers from the allocation time tracking if they were allocated 20 days ago
        containers_to_remove = [container_id for container_id, allocation_time in container_allocation_time.items() if allocation_time >= 20]
        for container_id in containers_to_remove:
            del container_allocation_time[container_id]
        
        # Add the containers with allocation time less than 20 days back to eligible_containers
        eligible_containers = pd.concat([eligible_containers, supply_df[supply_df["ContainerID"].isin(containers_to_remove)]])
        
        # Create a list to store allocated container IDs for this day
        allocated_container_ids = []
        
        for _, container_row in eligible_containers.iterrows():
            container_id = container_row["ContainerID"]
            
            # Check if the container was allocated within the last 20 days
            if container_id in container_allocation_time and container_allocation_time[container_id] < 20:
                continue
            
            capacity = container_row["Capacity (TEU)"]

            if daily_demand >= capacity:
                allocation_df.at[_, "Allocated (TEU)"] = capacity
                daily_demand -= capacity
                allocated_container_ids.append(container_id)  # Add allocated container ID to the list
                allocated_container_ids_month.append(container_id)
                
                # Update the allocation time for the container
                container_allocation_time[container_id] = 0
                
                # Set the shipping status of the container to 0
                allocation_df.at[_, "Shipping Status"] = 0
            else:
                allocation_df.at[_, "Allocated (TEU)"] = daily_demand
                daily_demand = 0
                allocated_container_ids.append(container_id)  # Add allocated container ID to the list
                allocated_container_ids_month.append(container_id)
                
                # Update the allocation time for the container
                container_allocation_time[container_id] = 0
                
                # Set the shipping status of the container to 0
                allocation_df.at[_, "Shipping Status"] = 0

            if daily_demand == 0:
                break
        
        # Increment the allocation time for all containers
        for container_id in container_allocation_time:
            container_allocation_time[container_id] += 1
        
        # Print detailed information of allocated containers for this day
        month = month_numbers[month_input]
        print(f"Detailed Information of Allocated Containers for {month}, Day {day}:")
        allocated_containers_info = allocation_df[allocation_df["ContainerID"].isin(allocated_container_ids)]
        print(allocated_containers_info)
        print("\n")
        
    # Print the container IDs that are being allocated for this month
    print(f"Container IDs Allocated for {month}:")
    print(allocated_container_ids_month)
    print("\n")
    
# Interactive input
month_input = int(input("Enter the month (e.g., '1-12'): "))
seasonality = float(input("Enter the seasonality factor: "))
market_trends = float(input("Enter the market trends factor: "))
temperature = float(input("Enter the temperature: "))
competitor_prices = float(input("Enter the competitor prices: "))

# Call the allocate_and_forecast function with user input
allocate_and_forecast(month, seasonality, market_trends, temperature, competitor_prices)


Enter the month (e.g., '1-12'): 1
Enter the seasonality factor: 0.01
Enter the market trends factor: 0.01
Enter the temperature: 20
Enter the competitor prices: 100
Forecasted Demand for 1: 6537.71 TEU
Detailed Information of Allocated Containers for Jan, Day 1:
    ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
5          C006              33                0        33.000000
68         C069              27                0        27.000000
76         C077              62                0        34.893744
80         C081              22                0        22.000000
184        C185              94                0        94.000000


Detailed Information of Allocated Containers for Jan, Day 2:
    ContainerID  Capacity (TEU)  Shipping Status  Allocated (TEU)
8          C009              45                0        45.000000
9          C010              97                0        97.000000
47         C048              57                0        57.000000
166        C16